# Launch TMDB Distributed Download Fleet

```
https://github.com/hudsonmendes/lambda-tmdb-distributed-downloader
MENDES, Hudson
14th May, 2020
London, UK
```

## Summary

The `lambda TMDB distributed downloader` is an **[AWS Lambda Function](https://aws.amazon.com/lambda/)** hooked to an **[AWS SQS Queue](https://aws.amazon.com/sqs/)**.

In order to launch our download fleet, we must then send messages to **SQS** and let our lambda function find them and start the download process.

This notebook reads the **[IMDB Titles Dataset]** in order to determine the download partitions (based in `year` and `initial` letter of the title), and sends messages for
each of those partitions.

## Environment

### Settings

In [18]:
lambda_name          = 'hudsonmendes-tmdb-downloader-lambda'
queue_name           = 'hudsonmendes-tmdb-downloader-queue'
datalake_bucket_name = 'hudsonmendes-datalake'
min_year             = 2000 

### Dependencies

In [2]:
%%bash
pip install -U pandas
pip install -r requirements.txt

Requirement already up-to-date: pandas in /Users/hudsonmendes/.pyenv/versions/3.6.10/lib/python3.6/site-packages (1.0.3)


### `tdd` deployment

We first must deploy/prepare our AWS infra-structure, and we do so by using our `cli`

In [3]:
!python tdd deploy \
    --lambda_name $lambda_name \
    --queue_name $queue_name \
    --datalake_bucket_name $datalake_bucket_name

Deploy, cleaning up environment
Deploy, saving dependencies locally
ERROR: awscli 1.18.46 has requirement botocore==1.15.46, but you'll have botocore 1.16.11 which is incompatible.
Deploy, adding code to ZIP
Deploy, ensure queue hudsonmendes-tmdb-downloader-queue
Deploy, ensure lambda hudsonmendes-tmdb-downloader-lambda
Deploy, link lambda hudsonmendes-tmdb-downloader-lambda to queue hudsonmendes-tmdb-downloader-queue
Deploy, cleaning up environment


### Imports

In [4]:
import os
import boto3
import json
import pandas as pd
from urllib.request import urlopen
from tqdm.notebook import tqdm
from tdd.pipeline import IMDbMovie

### Storage

In [5]:
IMDB_MOVIES_URL  = 'https://datasets.imdbws.com/title.basics.tsv.gz'
IMDB_MOVIES_PATH = '/tmp/title.basics.tsv.gz'

In [6]:
if not os.path.isfile(IMDB_MOVIES_PATH):
    with urlopen(IMDB_MOVIES_URL) as res:
        with open(IMDB_MOVIES_PATH, 'wb+') as out_file:
            out_file.write(res.read())
IMDB_MOVIES_PATH

'/tmp/title.basics.tsv.gz'

### AWS SQS

In [7]:
sqs = boto3.resource('sqs')
queue = sqs.get_queue_by_name(QueueName=queue_name)

## Data

### IMDB, Titles

In [8]:
df = pd.read_csv(IMDB_MOVIES_PATH, delimiter='\t', header=0)

/Users/hudsonmendes/.pyenv/versions/3.6.10/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
df = df[df.titleType == 'movie']

array(['short', 'movie', 'tvMovie', 'tvSeries', 'tvEpisode', 'tvShort',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame'], dtype=object)

In [9]:
df['initial'] = df['primaryTitle'].map(IMDbMovie.get_initial_from)

In [10]:
df['year'] = df['startYear'].map(IMDbMovie.get_year_from)

### Fleet Partitions

In [11]:
initials = set()
for initial in tqdm(df.initial.unique()):
    if initial and len(initial.strip()) == 2:
        initials.add(initial)
initials = sorted(list(initials))
pd.DataFrame(initials)

,0
0,00
1,01
2,02
3,03
4,04
...,...
1243,ZX
1244,ZY
1245,ZZ
1246,Z_


In [21]:
import math, datetime

years = set()
for year in tqdm(df.year.unique()):
    if year and \
            not math.isnan(year) and \
            year >= min_year and \
            year <= datetime.datetime.now().year: 
        years.add(int(year))
years = sorted(list(years))
pd.DataFrame(years)

,0
0,2000
1,2001
2,2002
3,2003
4,2004
5,2005
6,2006
7,2007
8,2008
9,2009


In [13]:
import math
partitions = set()
for year, initial in tqdm(df[['year', 'initial']].values):
    if not math.isnan(year):
        year = int(year)
        partitions.add((year, initial))
partitions = sorted(partitions)

In [14]:
len(partitions)

51472

## Launching

In [15]:
partition_iterator = iter(partitions)

### First Batch

Here we schedule just a few messages, to see if everything is alright.

In [16]:
messages_max = 3
messages_count = 0
for year, initial in partition_iterator:
    print((year, initial))
    message = { 'year': year, 'initial': initial }
    body = json.dumps(message)
    queue.send_message(MessageBody=body)
    messages_count += 1
    if messages_count >= messages_max:
        break

(1874, 'PA')
(1878, 'SA')
(1881, 'AT')


### Everything

In [17]:
for year, initial in partition_iterator:
    print((year, initial))
    message = { 'year': year, 'initial': initial }
    body = json.dumps(message)
    queue.send_message(MessageBody=body)

(1883, 'BU')
(1885, 'LH')
(1887, 'AD')
(1887, 'AN')
(1887, 'BA')
(1887, 'CA')
(1887, 'CH')
(1887, 'CO')
(1887, 'DO')
(1887, 'EL')
(1887, 'GN')
(1887, 'GO')
(1887, 'GU')
(1887, 'HO')
(1887, 'JU')
(1887, 'KA')
(1887, 'LE')
(1887, 'LI')
(1887, 'MA')
(1887, 'MU')
(1887, 'OR')
(1887, 'OS')
(1887, 'OX')
(1887, 'PI')
(1887, 'RA')
(1887, 'SL')
(1887, 'ST')
(1887, 'TI')
(1887, 'VI')
(1887, 'WO')
(1888, 'AC')
(1888, 'BR')
(1888, 'PF')
(1888, 'RO')
(1888, 'TR')
(1889, 'HY')
(1889, 'LE')
(1890, 'LO')
(1890, 'MO')
(1890, 'TR')
(1891, 'DI')
(1891, 'DU')
(1891, 'JE')
(1891, 'LA')
(1891, 'ME')
(1891, 'MO')
(1891, 'NE')
(1891, 'TW')
(1892, 'A ')
(1892, 'BO')
(1892, 'FE')
(1892, 'LE')
(1892, 'MA')
(1892, 'PA')
(1892, 'UN')
(1892, 'WR')
(1893, 'BL')
(1893, 'HO')
(1893, 'RA')
(1894, 'A ')
(1894, 'AL')
(1894, 'AM')
(1894, 'AN')
(1894, 'AR')
(1894, 'AT')
(1894, 'AU')
(1894, 'BA')
(1894, 'BE')
(1894, 'BO')
(1894, 'BU')
(1894, 'CA')
(1894, 'CH')
(1894, 'CO')
(1894, 'CR')
(1894, 'CU')
(1894, 'DA')
(1894, 'DI')

(1898, 'O ')
(1898, 'OB')
(1898, 'OF')
(1898, 'OL')
(1898, 'ON')
(1898, 'OP')
(1898, 'OR')
(1898, 'OS')
(1898, 'OU')
(1898, 'OX')
(1898, 'PA')
(1898, 'PE')
(1898, 'PH')
(1898, 'PI')
(1898, 'PL')
(1898, 'PO')
(1898, 'PR')
(1898, 'PU')
(1898, 'PY')
(1898, 'QA')
(1898, 'QU')
(1898, 'RA')
(1898, 'RE')
(1898, 'RH')
(1898, 'RI')
(1898, 'RM')
(1898, 'RO')
(1898, 'RU')
(1898, 'SA')
(1898, 'SC')
(1898, 'SE')
(1898, 'SH')
(1898, 'SI')
(1898, 'SK')
(1898, 'SL')
(1898, 'SM')
(1898, 'SN')
(1898, 'SO')
(1898, 'SP')
(1898, 'SQ')
(1898, 'SS')
(1898, 'ST')
(1898, 'SU')
(1898, 'SW')
(1898, 'TA')
(1898, 'TE')
(1898, 'TH')
(1898, 'TI')
(1898, 'TO')
(1898, 'TR')
(1898, 'TU')
(1898, 'TW')
(1898, 'TY')
(1898, 'UN')
(1898, 'US')
(1898, 'VA')
(1898, 'VE')
(1898, 'VI')
(1898, 'VO')
(1898, 'VR')
(1898, 'VY')
(1898, 'WA')
(1898, 'WE')
(1898, 'WH')
(1898, 'WI')
(1898, 'WO')
(1898, 'WR')
(1898, 'WY')
(1898, 'YV')
(1898, 'ZA')
(1898, 'ZD')
(1899, '10')
(1899, '1S')
(1899, '20')
(1899, '2N')
(1899, '33')
(1899, '43')

(1902, 'BA')
(1902, 'BE')
(1902, 'BF')
(1902, 'BI')
(1902, 'BL')
(1902, 'BO')
(1902, 'BR')
(1902, 'BU')
(1902, 'CA')
(1902, 'CE')
(1902, 'CH')
(1902, 'CI')
(1902, 'CL')
(1902, 'CO')
(1902, 'CR')
(1902, 'CS')
(1902, 'CU')
(1902, 'DA')
(1902, 'DE')
(1902, 'DI')
(1902, 'DO')
(1902, 'DR')
(1902, 'DU')
(1902, 'EA')
(1902, 'ED')
(1902, 'EE')
(1902, 'EF')
(1902, 'EG')
(1902, 'EI')
(1902, 'EJ')
(1902, 'EL')
(1902, 'EM')
(1902, 'EN')
(1902, 'EQ')
(1902, 'ER')
(1902, 'ET')
(1902, 'EV')
(1902, 'EX')
(1902, 'FA')
(1902, 'FE')
(1902, 'FI')
(1902, 'FL')
(1902, 'FO')
(1902, 'FR')
(1902, 'FU')
(1902, 'GA')
(1902, 'GE')
(1902, 'GI')
(1902, 'GO')
(1902, 'GR')
(1902, 'GU')
(1902, 'HA')
(1902, 'HE')
(1902, 'HI')
(1902, 'HM')
(1902, 'HO')
(1902, 'HS')
(1902, 'HU')
(1902, 'HY')
(1902, 'I ')
(1902, 'IC')
(1902, 'ID')
(1902, 'IL')
(1902, 'IN')
(1902, 'IR')
(1902, 'IS')
(1902, 'IT')
(1902, 'JA')
(1902, 'JE')
(1902, 'JI')
(1902, 'JO')
(1902, 'KA')
(1902, 'KE')
(1902, 'KI')
(1902, 'KO')
(1902, 'KR')
(1902, 'KY')

(1905, 'JE')
(1905, 'JI')
(1905, 'JO')
(1905, 'JU')
(1905, 'KA')
(1905, 'KE')
(1905, 'KI')
(1905, 'KO')
(1905, 'KU')
(1905, 'LA')
(1905, 'LE')
(1905, 'LI')
(1905, 'LL')
(1905, 'LO')
(1905, 'LU')
(1905, 'MA')
(1905, 'ME')
(1905, 'MI')
(1905, 'MO')
(1905, 'MR')
(1905, 'MU')
(1905, 'MY')
(1905, 'NA')
(1905, 'NE')
(1905, 'NI')
(1905, 'NO')
(1905, 'NY')
(1905, 'OB')
(1905, 'OD')
(1905, 'OH')
(1905, 'OL')
(1905, 'ON')
(1905, 'OO')
(1905, 'OP')
(1905, 'OR')
(1905, 'OS')
(1905, 'OT')
(1905, 'OU')
(1905, 'OV')
(1905, 'OX')
(1905, 'PA')
(1905, 'PE')
(1905, 'PH')
(1905, 'PI')
(1905, 'PL')
(1905, 'PO')
(1905, 'PR')
(1905, 'PU')
(1905, 'QU')
(1905, 'RA')
(1905, 'RE')
(1905, 'RI')
(1905, 'RO')
(1905, 'RU')
(1905, 'RY')
(1905, 'SA')
(1905, 'SC')
(1905, 'SE')
(1905, 'SH')
(1905, 'SI')
(1905, 'SK')
(1905, 'SL')
(1905, 'SM')
(1905, 'SN')
(1905, 'SO')
(1905, 'SP')
(1905, 'SQ')
(1905, 'ST')
(1905, 'SU')
(1905, 'SV')
(1905, 'SW')
(1905, 'TA')
(1905, 'TE')
(1905, 'TH')
(1905, 'TI')
(1905, 'TJ')
(1905, 'TO')

(1908, 'OF')
(1908, 'OG')
(1908, 'OH')
(1908, 'OL')
(1908, 'ON')
(1908, 'OP')
(1908, 'OR')
(1908, 'OS')
(1908, 'OT')
(1908, 'OU')
(1908, 'OV')
(1908, 'OX')
(1908, 'OY')
(1908, 'PA')
(1908, 'PE')
(1908, 'PH')
(1908, 'PI')
(1908, 'PL')
(1908, 'PO')
(1908, 'PR')
(1908, 'PS')
(1908, 'PU')
(1908, 'QU')
(1908, 'RA')
(1908, 'RE')
(1908, 'RH')
(1908, 'RI')
(1908, 'RO')
(1908, 'RU')
(1908, 'SA')
(1908, 'SC')
(1908, 'SE')
(1908, 'SF')
(1908, 'SH')
(1908, 'SI')
(1908, 'SK')
(1908, 'SL')
(1908, 'SM')
(1908, 'SO')
(1908, 'SP')
(1908, 'SQ')
(1908, 'ST')
(1908, 'SU')
(1908, 'SV')
(1908, 'SW')
(1908, 'TA')
(1908, 'TE')
(1908, 'TH')
(1908, 'TI')
(1908, 'TO')
(1908, 'TR')
(1908, 'TU')
(1908, 'TW')
(1908, 'TY')
(1908, 'UL')
(1908, 'UM')
(1908, 'UN')
(1908, 'UP')
(1908, 'UR')
(1908, 'US')
(1908, 'VA')
(1908, 'VE')
(1908, 'VI')
(1908, 'VL')
(1908, 'VO')
(1908, 'VU')
(1908, 'WA')
(1908, 'WE')
(1908, 'WH')
(1908, 'WI')
(1908, 'WO')
(1908, 'WR')
(1908, 'XI')
(1908, 'XY')
(1908, 'YA')
(1908, 'YE')
(1908, 'YO')

...

(2010, '70')
(2010, '71')
(2010, '72')
(2010, '74')
(2010, '75')
(2010, '76')
(2010, '77')
(2010, '78')
(2010, '79')
(2010, '7B')
(2010, '7D')
(2010, '7E')
(2010, '7L')
(2010, '7S')
(2010, '7T')
(2010, '7U')
(2010, '7X')
(2010, '8 ')
(2010, '80')
(2010, '81')
(2010, '82')
(2010, '83')
(2010, '84')
(2010, '85')
(2010, '86')
(2010, '87')
(2010, '88')
(2010, '89')
(2010, '8B')
(2010, '8E')
(2010, '8I')
(2010, '8M')
(2010, '8T')
(2010, '8U')
(2010, '8_')
(2010, '9 ')
(2010, '90')
(2010, '91')
(2010, '92')
(2010, '93')
(2010, '94')
(2010, '95')
(2010, '96')
(2010, '97')
(2010, '98')
(2010, '99')
(2010, '9B')
(2010, '9K')
(2010, '9T')
(2010, 'A ')
(2010, 'A2')
(2010, 'A3')
(2010, 'A4')
(2010, 'A7')
(2010, 'AA')
(2010, 'AB')
(2010, 'AC')
(2010, 'AD')
(2010, 'AE')
(2010, 'AF')
(2010, 'AG')
(2010, 'AH')
(2010, 'AI')
(2010, 'AJ')
(2010, 'AK')
(2010, 'AL')
(2010, 'AM')
(2010, 'AN')
(2010, 'AO')
(2010, 'AP')
(2010, 'AQ')
(2010, 'AR')
(2010, 'AS')
(2010, 'AT')
(2010, 'AU')
(2010, 'AV')
(2010, 'AW')

(2010, 'WM')
(2010, 'WO')
(2010, 'WP')
(2010, 'WR')
(2010, 'WS')
(2010, 'WT')
(2010, 'WU')
(2010, 'WW')
(2010, 'WY')
(2010, 'W_')
(2010, 'X ')
(2010, 'X1')
(2010, 'X3')
(2010, 'XA')
(2010, 'XB')
(2010, 'XC')
(2010, 'XD')
(2010, 'XE')
(2010, 'XF')
(2010, 'XG')
(2010, 'XI')
(2010, 'XL')
(2010, 'XM')
(2010, 'XO')
(2010, 'XP')
(2010, 'XR')
(2010, 'XS')
(2010, 'XT')
(2010, 'XU')
(2010, 'XV')
(2010, 'XX')
(2010, 'XY')
(2010, 'X_')
(2010, 'Y ')
(2010, 'Y2')
(2010, 'YA')
(2010, 'YB')
(2010, 'YD')
(2010, 'YE')
(2010, 'YG')
(2010, 'YH')
(2010, 'YI')
(2010, 'YK')
(2010, 'YL')
(2010, 'YM')
(2010, 'YN')
(2010, 'YO')
(2010, 'YP')
(2010, 'YR')
(2010, 'YS')
(2010, 'YU')
(2010, 'YV')
(2010, 'Z ')
(2010, 'ZA')
(2010, 'ZB')
(2010, 'ZD')
(2010, 'ZE')
(2010, 'ZG')
(2010, 'ZH')
(2010, 'ZI')
(2010, 'ZK')
(2010, 'ZL')
(2010, 'ZM')
(2010, 'ZN')
(2010, 'ZO')
(2010, 'ZP')
(2010, 'ZR')
(2010, 'ZS')
(2010, 'ZT')
(2010, 'ZU')
(2010, 'ZV')
(2010, 'ZW')
(2010, 'ZY')
(2010, 'ZZ')
(2010, '__')
(2011, '')
(2011, '00')
(

(2011, 'NG')
(2011, 'NH')
(2011, 'NI')
(2011, 'NL')
(2011, 'NM')
(2011, 'NN')
(2011, 'NO')
(2011, 'NP')
(2011, 'NQ')
(2011, 'NR')
(2011, 'NS')
(2011, 'NT')
(2011, 'NU')
(2011, 'NV')
(2011, 'NW')
(2011, 'NX')
(2011, 'NY')
(2011, 'O ')
(2011, 'OA')
(2011, 'OB')
(2011, 'OC')
(2011, 'OD')
(2011, 'OE')
(2011, 'OF')
(2011, 'OG')
(2011, 'OH')
(2011, 'OI')
(2011, 'OJ')
(2011, 'OK')
(2011, 'OL')
(2011, 'OM')
(2011, 'ON')
(2011, 'OO')
(2011, 'OP')
(2011, 'OR')
(2011, 'OS')
(2011, 'OT')
(2011, 'OU')
(2011, 'OV')
(2011, 'OW')
(2011, 'OX')
(2011, 'OY')
(2011, 'OZ')
(2011, 'P ')
(2011, 'P0')
(2011, 'P3')
(2011, 'P5')
(2011, 'P9')
(2011, 'PA')
(2011, 'PB')
(2011, 'PC')
(2011, 'PD')
(2011, 'PE')
(2011, 'PF')
(2011, 'PG')
(2011, 'PH')
(2011, 'PI')
(2011, 'PJ')
(2011, 'PK')
(2011, 'PL')
(2011, 'PM')
(2011, 'PN')
(2011, 'PO')
(2011, 'PR')
(2011, 'PS')
(2011, 'PT')
(2011, 'PU')
(2011, 'PV')
(2011, 'PW')
(2011, 'PY')
(2011, 'PZ')
(2011, 'Q ')
(2011, 'QA')
(2011, 'QI')
(2011, 'QN')
(2011, 'QP')
(2011, 'QR')

(2012, 'EA')
(2012, 'EB')
(2012, 'EC')
(2012, 'ED')
(2012, 'EE')
(2012, 'EF')
(2012, 'EG')
(2012, 'EH')
(2012, 'EI')
(2012, 'EJ')
(2012, 'EK')
(2012, 'EL')
(2012, 'EM')
(2012, 'EN')
(2012, 'EO')
(2012, 'EP')
(2012, 'EQ')
(2012, 'ER')
(2012, 'ES')
(2012, 'ET')
(2012, 'EU')
(2012, 'EV')
(2012, 'EW')
(2012, 'EX')
(2012, 'EY')
(2012, 'EZ')
(2012, 'F ')
(2012, 'F1')
(2012, 'F2')
(2012, 'F4')
(2012, 'F5')
(2012, 'FA')
(2012, 'FB')
(2012, 'FC')
(2012, 'FD')
(2012, 'FE')
(2012, 'FF')
(2012, 'FG')
(2012, 'FH')
(2012, 'FI')
(2012, 'FJ')
(2012, 'FK')
(2012, 'FL')
(2012, 'FM')
(2012, 'FN')
(2012, 'FO')
(2012, 'FP')
(2012, 'FR')
(2012, 'FS')
(2012, 'FT')
(2012, 'FU')
(2012, 'FW')
(2012, 'FX')
(2012, 'FY')
(2012, 'F_')
(2012, 'G ')
(2012, 'G0')
(2012, 'G3')
(2012, 'G6')
(2012, 'GA')
(2012, 'GB')
(2012, 'GC')
(2012, 'GD')
(2012, 'GE')
(2012, 'GF')
(2012, 'GG')
(2012, 'GH')
(2012, 'GI')
(2012, 'GJ')
(2012, 'GL')
(2012, 'GM')
(2012, 'GN')
(2012, 'GO')
(2012, 'GP')
(2012, 'GQ')
(2012, 'GR')
(2012, 'GS')

KeyboardInterrupt: 

### Evaluation

In [ ]:
df[df['tconst'] == 'tt9686708']